In [1]:
room_config = {
    "static_objects": [
        ["bed", "bedroom"],
        ["desk", "officeroom"],
        ["table", "livingroom"],
    ],
    "independent_objects": [
        ["tae", {"officeroom": 0.5, "livingroom": 0.5}],
        ["michael", {"bedroom": 0.5, "livingroom": 0.5}],
        ["vincent", {"bedroom": 0.5, "officeroom": 0.5}],
    ],
    "dependent_objects": [
        ["laptop", [["tae", 0.7], ["michael", 0.4], ["vincent", 0.1]]],
        ["phone", [["tae", 0.1], ["michael", 0.7], ["vincent", 0.4]]],
        ["headset", [["tae", 0.4], ["michael", 0.1], ["vincent", 0.9]]],
    ],
    "agent": {"bedroom": 0.333, "officeroom": 0.333, "livingroom": 0.333},
}


In [2]:
from room_env.envs.room2 import RoomEnv2

env = RoomEnv2(room_config=room_config)
env.objs

/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[StaticObject(bed, bedroom),
 StaticObject(desk, officeroom),
 StaticObject(table, livingroom),
 IndependentObject(tae, livingroom),
 IndependentObject(michael, bedroom),
 IndependentObject(vincent, bedroom),
 DependentObject(laptop, livingroom, None),
 DependentObject(phone, bedroom, None),
 DependentObject(headset, bedroom, None),
 Agent(agent, bedroom)]

In [3]:
env.rooms

{'livingroom': [StaticObject(table, livingroom),
  IndependentObject(tae, livingroom),
  DependentObject(laptop, livingroom, None)],
 'officeroom': [StaticObject(desk, officeroom)],
 'bedroom': [StaticObject(bed, bedroom),
  IndependentObject(michael, bedroom),
  IndependentObject(vincent, bedroom),
  DependentObject(phone, bedroom, None),
  DependentObject(headset, bedroom, None),
  Agent(agent, bedroom)]}

In [4]:
env.action_space

Discrete(3)

In [8]:
list(set([foo.location for foo in env.objs]))

['officeroom', 'livingroom', 'bedroom']

In [62]:
room_config["static_objects"][0]

['bed', 'bedroom']

In [ ]:
env.ob

In [63]:
room_config["independent_objects"][0]

['tae', {'officeroom': 0.5, 'livingroom': 0.5}]

In [70]:
obj = room_config["dependent_objects"][0]
obj

['laptop', [['tae', 0.7], ['michael', 0.4], ['vincent', 0.1]]]

In [73]:
[(io, prob) for io, prob in obj[1]]

[('tae', 0.7), ('michael', 0.4), ('vincent', 0.1)]

In [65]:
room_config["agent"]

{'bedroom': 0.333, 'officeroom': 0.333, 'livingroom': 0.333}

In [1]:
from room_env.envs.room2 import (
    Object,
    StaticObject,
    IndepdentObject,
    DependentObject,
    Agent,
)

bed = StaticObject("bed", "bedroom")
desk = StaticObject("desk", "officeroom")
table = StaticObject("table", "livingroom")

# independent objects
tae = IndepdentObject("tae", {"officeroom": 0.5, "livingroom": 0.5})
michael = IndepdentObject("michael", {"bedroom": 0.5, "livingroom": 0.5})
vincent = IndepdentObject("vincent", {"bedroom": 0.5, "officeroom": 0.5})

# dependent objects
laptop = DependentObject("laptop", [(tae, 0.7), (michael, 0.4), (vincent, 0.1)])
phone = DependentObject("phone", [(tae, 0.1), (michael, 0.7), (vincent, 0.4)])
headset = DependentObject("headset", [(tae, 0.4), (michael, 0.1), (vincent, 0.9)])

# agent
agent = Agent({"bedroom": 0.333, "officeroom": 0.333, "livingroom": 0.333})


/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
michael.name

'michael'

In [2]:
bed, desk, table, tae, michael, vincent, laptop, phone, headset, agent

(StaticObject(bed, bedroom),
 StaticObject(desk, officeroom),
 StaticObject(table, livingroom),
 IndependentObject(tae, livingroom),
 IndependentObject(michael, bedroom),
 IndependentObject(vincent, officeroom),
 DependentObject(laptop, bedroom, None),
 DependentObject(phone, bedroom, None),
 DependentObject(headset, officeroom, None),
 Agent(agent, livingroom))

In [26]:
laptop.type

'dependent'

In [3]:
bed.move()
desk.move()
table.move()

laptop.attach()
phone.attach()
headset.attach()

tae.move()
michael.move()
vincent.move()

laptop.move()
phone.move()
headset.move()

agent.move("bedroom")

In [4]:
bed, desk, table, tae, michael, vincent, laptop, phone, headset, agent

(StaticObject(bed, bedroom),
 StaticObject(desk, officeroom),
 StaticObject(table, livingroom),
 IndependentObject(tae, livingroom),
 IndependentObject(michael, bedroom),
 IndependentObject(vincent, bedroom),
 DependentObject(laptop, bedroom, None),
 DependentObject(phone, bedroom, None),
 DependentObject(headset, bedroom, IndependentObject(vincent, bedroom)),
 Agent(agent, bedroom))

In [5]:
bed.move()
desk.move()
table.move()

laptop.attach()
phone.attach()
headset.attach()

tae.move()
michael.move()
vincent.move()

laptop.move()
phone.move()
headset.move()

agent.move("officeroom")

In [6]:
bed, desk, table, tae, michael, vincent, laptop, phone, headset, agent

(StaticObject(bed, bedroom),
 StaticObject(desk, officeroom),
 StaticObject(table, livingroom),
 IndependentObject(tae, officeroom),
 IndependentObject(michael, livingroom),
 IndependentObject(vincent, officeroom),
 DependentObject(laptop, livingroom, IndependentObject(michael, livingroom)),
 DependentObject(phone, bedroom, None),
 DependentObject(headset, officeroom, IndependentObject(vincent, officeroom)),
 Agent(agent, officeroom))

In [7]:
bed.move()
desk.move()
table.move()

laptop.attach()
phone.attach()
headset.attach()

tae.move()
michael.move()
vincent.move()

laptop.move()
phone.move()
headset.move()

agent.move("livingroom")

In [8]:
bed, desk, table, tae, michael, vincent, laptop, phone, headset, agent

(StaticObject(bed, bedroom),
 StaticObject(desk, officeroom),
 StaticObject(table, livingroom),
 IndependentObject(tae, livingroom),
 IndependentObject(michael, livingroom),
 IndependentObject(vincent, officeroom),
 DependentObject(laptop, livingroom, IndependentObject(michael, livingroom)),
 DependentObject(phone, bedroom, None),
 DependentObject(headset, officeroom, IndependentObject(vincent, officeroom)),
 Agent(agent, livingroom))